In [1]:
from Crypto import Random
from Crypto.Cipher import AES
import base64
from hashlib import md5
import requests
from bs4 import BeautifulSoup

BLOCK_SIZE = 16
KEY = b"k8B$B@0L8D$tDYHGmRg98sQ7!%GOEGOX27T"

class TwistMoe():
    sitename = 'twist.moe'
    QUALITIES = ['360p', '480p', '720p', '1080p']
    _api_url = "https://twist.moe/api/anime/{}/sources"

    def __init__(self, url=None):
        self.url = url
    
    @classmethod
    def search(self, query):
        r = requests.get('https://twist.moe')
        soup = BeautifulSoup(r.text, 'html.parser')
        all_anime = soup.select_one('nav.series').select('li')
        animes = []
        for anime in all_anime:
            title = anime.find('span').contents[0].strip()
            if query in title.lower():
                animes.append({
                    "title":title,
                    "url":'https://twist.moe' + anime.find('a')['href']
                })
        return animes

    def get_data(self):
        anime_name = self.url.split('/a/')[-1].split('/')[0]
        url = self._api_url.format(anime_name)
        episodes = requests.get(
            url,
            headers={
                'x-access-token': '1rj2vRtegS8Y60B3w3qNZm5T2Q0TN2NR'
            }
        )
        episodes = episodes.json()
        self.title = anime_name
        episode_urls = ['https://eu1.twist.moe' +
                        decrypt(episode['source'].encode('utf-8'), KEY).decode('utf-8')
                        for episode in episodes]

        self._episode_urls = [(i+1, episode_url) for i, episode_url in enumerate(episode_urls)]
        self._len = len(self._episode_urls)

        return self._episode_urls



# From stackoverflow https://stackoverflow.com/questions/36762098/how-to-decrypt-password-from-javascript-cryptojs-aes-encryptpassword-passphras
def pad(data):
    length = BLOCK_SIZE - (len(data) % BLOCK_SIZE)
    return data + (chr(length)*length).encode()


def unpad(data):
    return data[:-(data[-1] if type(data[-1]) == int else ord(data[-1]))]


def bytes_to_key(data, salt, output=48):
    # extended from https://gist.github.com/gsakkis/4546068
    assert len(salt) == 8, len(salt)
    data += salt
    key = md5(data).digest()
    final_key = key
    while len(final_key) < output:
        key = md5(key + data).digest()
        final_key += key
    return final_key[:output]


def decrypt(encrypted, passphrase):
    encrypted = base64.b64decode(encrypted)
    assert encrypted[0:8] == b"Salted__"
    salt = encrypted[8:16]
    key_iv = bytes_to_key(passphrase, salt, 32+16)
    key = key_iv[:32]
    iv = key_iv[32:]
    aes = AES.new(key, AES.MODE_CBC, iv)
    return unpad(aes.decrypt(encrypted[16:]))


In [2]:
tw = TwistMoe()

In [3]:
allan = tw.search('gintama')
allan

[{'title': 'Gintama', 'url': 'https://twist.moe/a/gintama/first'},
 {'title': 'Gintama: Jump Festa 2015 Special',
  'url': 'https://twist.moe/a/gintama-jump-festa-2015-special/first'}]

In [4]:
tw = TwistMoe('gintama')

In [5]:
tw.get_data()

[(1,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 001 [480p].mp4'),
 (2,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 002 [480p].mp4'),
 (3,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 003 [480p].mp4'),
 (4,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 004 [480p].mp4'),
 (5,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 005 [480p].mp4'),
 (6,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 006 [480p].mp4'),
 (7,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 007 [480p].mp4'),
 (8,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 008 [480p].mp4'),
 (9,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 009 [480p].mp4'),
 (10,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 010 [480p].mp4'),
 (11,
  'https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 011 [480p].mp4'),
 (12,
  'https://eu1.twist.moe/anime/gint

In [6]:
from IPython.display import HTML, Javascript

In [7]:
HTML(data='''
    <div id="video">
        <video
            src="{}"
            width="{}"
            id="vid"
            controls>
        </video>
    </div>
    '''.format("https://eu1.twist.moe/anime/gintama/[HorribleSubs] Gintama - 356 [1080p].mp4",800))

In [8]:
Javascript('''
        var vid = document.querySelector('#vid');

        vid.addEventListener('click',()=>{ 
            vid.paused ? vid.play() : vid.pause();
        });

        vid.addEventListener('dblclick',()=>{
            !(document.webkitIsFullScreen) ? vid.webkitRequestFullScreen() : document.webkitExitFullscreen();
        });
        ''')

<IPython.core.display.Javascript object>

In [9]:
from difflib import SequenceMatcher

class Similar_search():
    """
        
        
    """
    def __init__(self, query, orig, thresh=0.6):
        self.query = query
        self.orig = orig
        self.thresh = thresh
    def __str__(self):
        return str(self.similarity)
    
    def __repr__(self):
        return str(self.similarity)
    
    def __lt__(self, other):
        return (self.score < other.score)
    
    def __eq__(self, other):
        return (self.score == other.score)
    
    @property
    def score(self):
        return SequenceMatcher(None, self.query, self.orig).ratio()
    
    @property
    def similarity(self):
        if self.query in self.orig:
            return True
        return (SequenceMatcher(None, self.query, self.orig).ratio() >= (self.thresh))



In [10]:
s = Similar_search("sss","s",0.9)

In [11]:
w = Similar_search("s","s")

In [12]:
w.score, s.score

(1.0, 0.5)

In [13]:
s > w, s<w, s==w, s!=w

(False, True, False, True)

In [14]:
s,bool(s)

(False, True)

In [15]:
if (bool(s)):
    print(w)

True


In [16]:
episodes = requests.get(
            "https://twist.moe/api/anime/gintama/sources",
            headers={
                'x-access-token': '1rj2vRtegS8Y60B3w3qNZm5T2Q0TN2NR'
            }
        )
episodes = episodes.json()

In [17]:
animelist = requests.get(
            "https://twist.moe/api/anime",
            headers={
                'x-access-token': '1rj2vRtegS8Y60B3w3qNZm5T2Q0TN2NR'
            }
        )
animelist = animelist.json()

In [18]:
x = ((decrypt(ep['source'],KEY)) for ep in episodes)

In [19]:
animeinfo = requests.get(
            "https://twist.moe/api/anime/{}".format('gintama'),
            headers={
                'x-access-token': '1rj2vRtegS8Y60B3w3qNZm5T2Q0TN2NR'
            }
        )
animeinfo = animeinfo.json()

In [20]:
episodes = requests.get(
            "https://twist.moe/api/anime/{}/sources".format('gintama'),
            headers={
                'x-access-token': '1rj2vRtegS8Y60B3w3qNZm5T2Q0TN2NR'
            }
        )
episodes = episodes.json()

In [21]:
animex = animelist[92]
print(animex.keys())

for anime in animelist:
    if (bool(anime['hidden'])):
        print(anime['slug']['slug'])

dict_keys(['ongoing', 'hb_id', 'season', 'slug', 'created_at', 'hidden', 'alt_title', 'updated_at', 'id', 'mal_id', 'title'])
basilisk-kouga-ninpou-chou
blood
hunter-x-hunter
inuyasha
rwby
nodame-cantabile-finale
suzuka
romeo-x-juliet
cardfight-vanguard-g-girs-crisis-hen
nana
hetalia-axis-powers
shiyan-pin-jiating-frankenstein-family
slayers-evolution-r
slayers-try
heroic-age


In [22]:
basilisk = TwistMoe('basilisk-kouga-ninpou-chou')

In [23]:
basilisk.get_data()

[]

In [24]:
import anime_downloader

In [25]:
twist = anime_downloader.get_anime_class('twist.moe')

In [26]:
s = twist('gintama')

In [27]:
s


Site: twist.moe
Anime: gintama
Episode count: 357

In [28]:
import pythontwist

In [29]:
tw = pythontwist.get_anime_class('twist.moe')

In [32]:
x.search('pok')

[<SearchResult Title: Pokemon: The Origin URL: https://twist.moe/a/pokemon-the-origin/first>,
 <SearchResult Title: Hajime no Ippo URL: https://twist.moe/a/hajime-no-ippo/first>,
 <SearchResult Title: Akkun no Kanojo URL: https://twist.moe/a/akkun-no-kanojo/first>,
 <SearchResult Title: Alice to Zouroku URL: https://twist.moe/a/alice-to-zouroku/first>,
 <SearchResult Title: Asagao to Kase-san. URL: https://twist.moe/a/asagao-to-kase-san/first>]

In [33]:
x.search('07')

[<SearchResult Title: 07-Ghost URL: https://twist.moe/a/07-ghost/first>,
 <SearchResult Title: Steins;Gate 0 URL: https://twist.moe/a/steins-gate-0/first>,
 <SearchResult Title: Berserk (2016) URL: https://twist.moe/a/berserk-2016/first>,
 <SearchResult Title: Berserk (2017) URL: https://twist.moe/a/berserk-2017/first>,
 <SearchResult Title: Fairy Tail (2014) URL: https://twist.moe/a/fairy-tail-2014/first>]

In [34]:
stgate = tw('steins-gate-0')

In [35]:
x =stgate.get_data()

In [50]:
HTML(data='''
    <div id="video2">
        <video
            src="{}"
            width="{}"
            id="vid2"
            controls>
        </video>
    </div>
'''.format(x[14][1],800))

In [51]:
Javascript('''
        var vid2 = document.querySelector('#vid2');

        vid2.addEventListener('click',()=>{ 
            vid2.paused ? vid2.play() : vid2.pause();
        });

        vid2.addEventListener('dblclick',()=>{
            !(document.webkitIsFullScreen) ? vid2.webkitRequestFullScreen() : document.webkitExitFullscreen();
        });
        ''')

<IPython.core.display.Javascript object>